---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [22]:
import pandas as pd
import numpy as np

def get_dataframes():
    # get energy data from Energy Indicators.xl
    energy_excel_file = pd.ExcelFile('Energy Indicators.xls')
    energy = energy_excel_file.parse('Energy')
    # get rid of first two columns
    energy = energy.drop('Unnamed: 0', 1)
    energy = energy.drop('Unnamed: 1', 1)
    # get rid of footer
    energy = energy.drop(energy.index[243:])
    # get rid of header
    energy = energy.drop(energy.index[0:16])    
    # change the column labels
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    # For all countries which have missing data (e.g. data with "...") make sure this is reflected as np.NaN values.
    energy = energy.replace('...', np.NaN)   
    # Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule). 
    gj = 1000000
    energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x*gj)

    #note that column C has the country names with footnote numbers while column B has no footnotes
    # if you read the C column in 1 then remove any digits in the country column   
    energy['Country'] = energy['Country'].str.strip('0123456789')    
    
    # Rename list of countries (for use in later questions):    
    energy = energy.replace('Republic of Korea', 'South Korea')
    energy = energy.replace("United States of America", "United States")
    energy = energy.replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")
    energy = energy.replace("China, Hong Kong Special Administrative Region", "Hong Kong")   
         
    # remove the part ' (...)' form all countries that have paranthesis, make 
    # sure the result has no trailing space
    energy['Country'] = energy['Country'].str.replace(r"\(.*\)","")    
    energy['Country'] = energy['Country'].str.strip()    
        
    # get GDP data from the file world_bank.csv
    GDP = pd.read_csv('world_bank.csv')
    # get rid of header
    GDP = GDP.drop(GDP.index[0:4])
    GDP = GDP.drop(GDP.ix[:,'Unnamed: 2':'Unnamed: 49'].head(0).columns, axis=1)
    GDP = GDP.drop('World Development Indicators', 1)
    # change the column labels
    GDP.columns = ['Country', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']    

    GDP = GDP.replace("Korea, Rep.", "South Korea")
    GDP = GDP.replace("Iran, Islamic Rep.", "Iran")
    GDP = GDP.replace("Hong Kong SAR, China", "Hong Kong")
    
    # get the Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology
    ScimEn_excel_file = pd.ExcelFile('scimagojr-3.xlsx')
    ScimEn = ScimEn_excel_file.parse('Sheet1')

    return energy, GDP, ScimEn 


def answer_one():
    energy, GDP, ScimEn = get_dataframes()    
    # Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). 
    df = pd.merge(ScimEn, energy, on = ['Country'])
    df = pd.merge(df, GDP, on = ['Country'])
    df = df.set_index('Country')
    # use only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).  
    df = df[df.Rank < 16]
   
    return df 

answer_one().head()



,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.57098,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.28868,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [2]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [21]:
import numpy as np

def get_dataframes():
    # get energy data from Energy Indicators.xl
    energy_excel_file = pd.ExcelFile('Energy Indicators.xls')
    energy = energy_excel_file.parse('Energy')
    # get rid of first two columns
    energy = energy.drop('Unnamed: 0', 1)
    energy = energy.drop('Unnamed: 1', 1)
    # get rid of footer
    energy = energy.drop(energy.index[243:])
    # get rid of header
    energy = energy.drop(energy.index[0:16])    
    # change the column labels
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    # For all countries which have missing data (e.g. data with "...") make sure this is reflected as np.NaN values.
    energy = energy.replace('...', np.NaN)   
    # Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule). 
    gj = 1000000
    energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x*gj)

    #note that column C has the country names with footnote numbers while column B has no footnotes
    # if you read the C column in 1 then remove any digits in the country column   
    energy['Country'] = energy['Country'].str.strip('0123456789')    
    
    # Rename list of countries (for use in later questions):    
    energy = energy.replace('Republic of Korea', 'South Korea')
    energy = energy.replace("United States of America", "United States")
    energy = energy.replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")
    energy = energy.replace("China, Hong Kong Special Administrative Region", "Hong Kong")   
         
    # remove the part ' (...)' form all countries that have paranthesis
    energy['Country'] = energy['Country'].str.replace(r"\(.*\)","")    
    # make sure the result has no trailing space
    energy['Country'] = energy['Country'].str.strip()    
        
    # get GDP data from the file world_bank.csv
    GDP = pd.read_csv('world_bank.csv')
    # get rid of header
    GDP = GDP.drop(GDP.index[0:4])
    GDP = GDP.drop(GDP.ix[:,'Unnamed: 2':'Unnamed: 49'].head(0).columns, axis=1)
    GDP = GDP.drop('World Development Indicators', 1)
    # change the column labels
    GDP.columns = ['Country', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']    

    GDP = GDP.replace("Korea, Rep.", "South Korea")
    GDP = GDP.replace("Iran, Islamic Rep.", "Iran")
    GDP = GDP.replace("Hong Kong SAR, China", "Hong Kong")
    
    # get the Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology
    ScimEn_excel_file = pd.ExcelFile('scimagojr-3.xlsx')
    ScimEn = ScimEn_excel_file.parse('Sheet1')

    return energy, GDP, ScimEn 



def answer_two():    
    Energy, GDP, ScimEn = get_dataframes()   
    # outer
    outer = pd.merge(Energy, ScimEn, how='outer', on = ['Country'])
    outer = pd.merge(outer,  GDP,    how='outer', on = ['Country'])
    # inner
    inner = pd.merge(ScimEn, Energy, how='inner', on = ['Country'])
    inner = pd.merge(inner,  GDP,    how='inner', on = ['Country'])
    
    return len(outer) - len(inner)


answer_two()

156

### Question 3 (6.6%)
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [23]:
def answer_three():
    Top15 = answer_one()    
    avgGDP = Top15[["2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015"]].mean(axis=1)
    return avgGDP

answer_three().head()

Country
China                 6.348609e+12
United States         1.536434e+13
Japan                 5.542208e+12
United Kingdom        2.487907e+12
Russian Federation    1.565459e+12
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [5]:
def answer_four():
    Top15 = answer_one()
    
    # from answer three, sort and get 6th largest
    sixth = answer_three().sort_values(ascending=False).index[5]
    
    # find how much the GDP changed over the 10 year span
    return Top15.loc[sixth, '2015'] - Top15.loc[sixth, '2006']


answer_four()


246702696075.3999

### Question 5 (6.6%)
What is the mean energy supply per capita?

*This function should return a single number.*

In [6]:
def answer_five():
    Top15 = answer_one()    
    meanEnergy = Top15['Energy Supply per Capita'].mean(axis=0)
    #meanEnergy = Top15['Energy Supply per Capita']
    return meanEnergy

answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [7]:
def answer_six():
    Top15 = answer_one()
    ren = [Top15['% Renewable'].sort_values(ascending=False).index[0], Top15['% Renewable'].sort_values(ascending=False)[0]]  
    return tuple(ren)

answer_six()


('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [8]:
def answer_seven():
    Top15 = answer_one()
    ratio = Top15[['Self-citations', 'Citations']]
    ratio['Ratio'] = ratio['Self-citations'] / ratio['Citations'] 
    ratio = (ratio['Ratio'].sort_values(ascending=False).index[0] , ratio['Ratio'].sort_values(ascending=False)[0] )
    return tuple(ratio)

answer_seven()


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [9]:
def answer_eight():
    Top15 = answer_one()
    Top15['Est_Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15 = Top15['Est_Population'].sort_values(ascending=False).index[2]
    
    
    return Top15

answer_eight()

'United States'

### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [10]:
def answer_nine():
    Top15 = answer_one()
    
    # Create Population Estimate from Energy Supply / Energy Supply per Capita: 
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita'] 
    
    # Create Documents per Capita from  Citable documents and Population Estimate
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['Population Estimate'] 

    # Perform pearson correlation
    Top15 = Top15['Energy Supply per Capita'].corr(Top15['Citable docs per Capita'])
    
    return Top15


answer_nine()

0.79400104354429424

In [11]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [12]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [24]:
def answer_ten():
    Top15 = answer_one()
    
    # get the median % Renewable value for all countries in the top 15
    median = Top15['% Renewable'].mean(axis=0)

    Top15['Median'] = np.where(Top15['% Renewable'] > median, 1, 0)
    
    #return median
    HighRenew = Top15[['% Renewable', 'Median']]
    
    HighRenew =  (HighRenew.sort_values(by= '% Renewable', ascending=False))
    return HighRenew['% Renewable']

answer_ten().head()


Country
Brazil    69.64803
Canada    61.94543
Spain     37.96859
Italy     33.66723
China     19.75491
Name: % Renewable, dtype: float64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [14]:
def answer_eleven():
    Top15 = answer_one()
    
    # get population estimate like before
    Top15['Est_Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    # reset index so we can use Counrty
    Top15 = Top15.reset_index()
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}  
    # this with help from forums:    
    Top15['Continent'] = [ContinentDict[country] for country in Top15['Country']]
    Continent_Populations = Top15.set_index('Continent').groupby(level=0)['Est_Population'].agg({'size': np.size, 'sum': np.sum, 'mean': np.mean,'std': np.std})
    Continent_Populations = Continent_Populations[['size', 'sum', 'mean', 'std']]
    return Continent_Populations


answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [25]:
import pandas as pd
import numpy as np


def answer_twelve():
    Top15 = answer_one()
    
    # reset index so we can use Counrty
    Top15 = Top15.reset_index()
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}    

    # this with help from forums:    
    Top15['% Renewable'] = [ContinentDict[country] for country in Top15['Country']]
    #MultiIndex_of_Continents = Top15.set_index('Continent').groupby(level=0)['Est_Population'].agg({'size': np.size, 'sum': np.sum, 'mean': np.mean,'std': np.std})
    #MultiIndex_of_Continents = MultiIndex_of_Continents[['size', 'sum', 'mean', 'std']]
    
#    return MultiIndex_of_Continents
    return Top15




answer_twelve().head()

,Country,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,...,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
1,United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,...,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
2,Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,...,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
3,United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,...,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
4,Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,...,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [26]:
import pandas as pd
import numpy as np
import locale

def answer_thirteen():
    Top15 = answer_one()
    
    # get population estimate like before
    Top15['Est_Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    #Top15['Est_Population'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).astype(float)
    
    #locale.setlocale(locale.LC_ALL, 'en_US.utf8')
    # sz = "{:0,}".format(1.2345)

    map_str = []
    for num in Top15['Est_Population']:
        #"{0:,f}".format(num)
        map_str.append(locale.format('%.7f', num, grouping=True).rstrip('0').rstrip('.'))
        #map_str.append(locale.format('%,F', num, grouping=True))
        #map_str = "{:0,}".format(map_str)
        
    Top15['Est_Population_str'] = map_str  


    return Top15['Est_Population_str']  
    
answer_thirteen().head()

Country
China                 1367645161.2903225
United States          317615384.6153846
Japan                  127409395.9731544
United Kingdom          63870967.7419355
Russian Federation             143500000
Name: Est_Population_str, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [17]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [18]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!